## 自动求导
正如我们在2.4节中所说的那样，求导是几乎所有深度学习优化算法的关键步骤。虽然求导的计算很简单，只需要一些基本的微积分，但对于复杂的模型，手工进行更新是一件很痛苦的事情（而且经常容易出错）。

深度学习框架通过自动计算导数，即 自动求导 （automatic differentiation），来加快这项工作。实际中，根据我们设计的模型，系统会构建一个 计算图 （computational graph），来跟踪计算是哪些数据通过哪些操作组合起来产生输出。自动求导使系统能够随后反向传播梯度。 这里，反向传播（backpropagate）只是意味着跟踪整个计算图，填充关于每个参数的偏导数。

## 一个简单的例子
作为一个演示例子，(**假设我们想对函数 $y = 2\mathbf{x}^{\top}\mathbf{x}$关于列向量 $\mathbf{x}$求导**)。首先，我们创建变量 `x` 并为其分配一个初始值。

In [6]:
import paddle

x = paddle.arange(4,dtype=paddle.float32)
x

Tensor(shape=[4], dtype=float32, place=Place(cpu), stop_gradient=True,
       [0., 1., 2., 3.])

[**在我们计算$y$关于$\mathbf{x}$的梯度之前，我们需要一个地方来存储梯度。**]
重要的是，我们不会在每次对一个参数求导时都分配新的内存。因为我们经常会成千上万次地更新相同的参数，每次都分配新的内存可能很快就会将内存耗尽。注意，标量函数关于向量$\mathbf{x}$的梯度是向量，并且与$\mathbf{x}$具有相同的形状。

In [2]:
x.stop_gradient = False 
x

Tensor(shape=[4], dtype=float32, place=Place(cpu), stop_gradient=False,
       [0., 1., 2., 3.])

In [3]:
y = 2 * paddle.dot(x, x)
y

Tensor(shape=[1], dtype=float32, place=Place(cpu), stop_gradient=False,
       [28.])

In [4]:
y.backward()
x.grad # 这里已经是完成了反向传播之后，就会对张量里面每一个元素进行梯度求导

D:\miniforge3\envs\paddle_env\lib\site-packages\paddle\fluid\dygraph\varbase_patch_methods.py:523: UserWarning: 
tensor.grad will return the tensor value of the gradient. This is an incompatible upgrade for tensor.grad API.  It's return type changes from numpy.ndarray in version 2.0 to paddle.Tensor in version 2.1.0.  If you want to get the numpy value of the gradient, you can use :code:`x.grad.numpy()` 
  warnings.warn(warning_msg)


Tensor(shape=[4], dtype=float32, place=Place(cpu), stop_gradient=False,
       [0. , 4. , 8. , 12.])

函数 $y = 2\mathbf{x}^{\top}\mathbf{x}$ 关于$\mathbf{x}$ 的梯度应为$4\mathbf{x}$。让我们快速验证我们想要的梯度是否正确计算。

In [5]:
x.grad == 4 * x

Tensor(shape=[4], dtype=bool, place=Place(cpu), stop_gradient=False,
       [True, True, True, True])

In [6]:
# 在默认情况下，PaddlePaddle会累积梯度，我们需要清除之前的值，所以最后全为1
x.clear_gradient()
y = paddle.sum(x)
y.backward()
x.grad

Tensor(shape=[4], dtype=float32, place=Place(cpu), stop_gradient=False,
       [1., 1., 1., 1.])